In [1]:
#! pip install praw

In [8]:
import pandas as pd
import requests as r
import sqlalchemy
import time
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import pprint
import praw
import re
#from config import client_id, client_secret
pd.options.display.max_columns = None

In [9]:
reddit = praw.Reddit(
  client_id = "L4CR_Ba3FnFGLg",
  client_secret = "D20nZIAi09ASSKcJPDyYeEPW31r7SQ",
  user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.75"
)

In [10]:
df = []
for post in reddit.subreddit('wallstreetbets').hot(limit=500):
    content = {
        "title" : post.title,
        "text" : post.selftext
    }
    df.append(content)
df = pd.DataFrame(df)

In [11]:
df

,title,text
0,"What Are Your Moves Tomorrow, April 14, 2021",Your daily trading discussion thread. Please k...
1,WSB loves Gain / Loss / YOLO posts. Rule Review.,"WSB is all about making high risk, high reward..."
2,GME - It's all coming together,"Apes, I come to you as a fellow primate with e..."
3,Kevin O'Leary Live Now thinks a second kick of...,
4,AFTER LOSING MY JOB. I STARTED A YEAR AGO WITH...,
...,...,...
242,MELVIN's LONG PORTFOLIO IS UP 7.62% IN Q1 2021...,"Hey guys,\n\n**by now we have all seen the hea..."
243,I just keep buying more GME. I have no idea wh...,
244,PSFE: An undervalued play with relatively low ...,&#x200B;\n\nhttps://preview.redd.it/h0hmwro2td...
245,Apes that missed getting into GME early,


In [12]:
# File to Load
file_to_load = "reddit_wsb.csv"

# Read Purchasing File and store into Pandas data frame
reddit_wsb_df = pd.read_csv(file_to_load)

In [13]:
reddit_wsb_df.sort_values(by=['comms_num'], inplace=True, ascending=False)
reddit_wsb_df.head()

,title,score,id,url,comms_num,created,body,timestamp
39993,"GME Megathread for March 10, 2021",47667,m1xhlc,https://www.reddit.com/r/wallstreetbets/commen...,93268,1.615412e+09,HOLYMOLYHOLYMOLYHOLYMOLYHOLYMOLYHOLYMOLYHOLYMO...,2021-03-10 23:30:11
17095,"Daily Discussion Thread for January 29, 2021",12288,l7ptxi,https://www.reddit.com/r/wallstreetbets/commen...,89050,1.611947e+09,Your daily trading discussion thread. Please k...,2021-01-29 21:00:15
35110,"GME Megathread for Wednesday, February 24th, 2021",28975,lre2k6,https://www.reddit.com/r/wallstreetbets/commen...,83094,1.614205e+09,Enjoy this thread overnight. New one comes at ...,2021-02-25 00:24:58
17249,"Daily Discussion Thread for January 29, 2021 -...",7348,l7wqsm,https://www.reddit.com/r/wallstreetbets/commen...,80926,1.611966e+09,Your daily trading discussion thread. Please ...,2021-01-30 02:23:51
17313,GME What About Second Breakfast Club Megathread,29356,l7v9o8,https://www.reddit.com/r/wallstreetbets/commen...,80586,1.611963e+09,NaN,2021-01-30 01:26:04


In [14]:
stocks = ['CVX', 'OXY', 'BP', 'XOM', 'RDS-B', 'CL=F', '^GSPC', 'GC=F']
names = ['Chevron', 'Oxy', 'BP', 'ExxonMobil', 'Shell', 'Oil Futures', 'S&P 500', 'Gold']

In [15]:
def data_retrieve(tickers, names, syear, smonth, sday, eyear, emonth, eday):
    start = datetime.datetime(syear, smonth, sday) # Select the start date (yyyy,m,d)
    end = datetime.datetime(eyear, emonth, eday) # Select the end date (yyyy,m,d)
    data = yf.download(tickers, start=start, end=end)
    data_fix = data.reset_index()
    data_fix.insert(0,'Name', names)
    data_fix.insert(1,'Ticker', tickers)
    return data_fix

In [16]:
master_df = data_retrieve(stocks[0], names[0], 2000, 1, 1, 2020, 12, 31)
master_df

[*********************100%***********************]  1 of 1 completed


,Name,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,Chevron,CVX,2000-01-03,42.937500,42.937500,41.281250,41.812500,19.360474,4387600
1,Chevron,CVX,2000-01-04,41.468750,42.062500,41.250000,41.812500,19.360474,3702400
2,Chevron,CVX,2000-01-05,41.531250,43.281250,41.531250,42.562500,19.707745,5567600
3,Chevron,CVX,2000-01-06,42.656250,44.593750,42.656250,44.375000,20.546988,4353400
4,Chevron,CVX,2000-01-07,45.000000,45.437500,44.500000,45.156250,20.908724,4487400
...,...,...,...,...,...,...,...,...,...
5278,Chevron,CVX,2020-12-23,85.000000,86.660004,85.000000,85.690002,84.495621,7223100
5279,Chevron,CVX,2020-12-24,85.930000,85.930000,84.809998,85.330002,84.140633,3335600
5280,Chevron,CVX,2020-12-28,85.610001,86.629997,84.620003,84.900002,83.716629,8051900
5281,Chevron,CVX,2020-12-29,85.260002,85.720001,84.050003,84.610001,83.430672,7670800
